In [1]:

#import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
#import numpy as np
from selenium.webdriver.support.ui import Select

s = Service(ChromeDriverManager().install())
option = webdriver.ChromeOptions()
option.add_argument('headless')
#option.add_argument("--window-size=1920,1080")  # Adjust window size
#option.add_argument("--disable-extensions")  # Disable Chrome extensions

#path = "C:\Users\dillonk\.cache\selenium\chromedriver\win64\124.0.6367.207\chromedriver.exe"
driver = webdriver.Chrome(service = s, options=option)


In [2]:


from selenium.common.exceptions import TimeoutException, WebDriverException

url_oddstrader = "https://www.oddstrader.com/ufc/"

try:
    driver.get(url_oddstrader)
except WebDriverException as e:
    print(f"An error occurred while loading the page: {e}")
    # Perform appropriate error handling or logging here

try:
    ot_all_players_list = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="teamName blueHover"]'))
    )

    day_of_game = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="generalDay"]'))
    )

    ot_away_players_list = []
    ot_home_players_list = []

    def gameday_checker(day_element):
        gamedays = {"SUN", "MON", "TUE", "WED", "THU", "FRI", "SAT"}
        return any(day in day_element.text for day in gamedays)

    player_index = 0
    # Iterate over the days to filter and collect player names
    for day_element in day_of_game:
        if gameday_checker(day_element):
            if player_index < len(ot_all_players_list) - 1:
                ot_away_players_list.append(ot_all_players_list[player_index].text)
                ot_home_players_list.append(ot_all_players_list[player_index + 1].text)
            player_index += 2  # Corrected the incrementation from 2 to 1

    # Print the lists of away and home players
    print("Away Players:", ot_away_players_list)
    print("Home Players:", ot_home_players_list)

except TimeoutException as te:
    print("Timeout occurred while waiting for elements:", te)
    # Log page source for debugging



Timeout occurred while waiting for elements: Message: 



In [3]:
#ODDSTRADER (BEST ODDS LISTED): 

ot_all_odds_list = []
ot_best_home_odds_list = []
ot_best_away_odds_list = []


ot_all_odds_list = WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.XPATH, '//span[@class="best-line"]'))
)

i = 0
while i < len(ot_away_players_list) + len (ot_home_players_list):
    if i % 2 == 0:
        ot_best_away_odds_list.append(ot_all_odds_list[i].text)
    elif i % 2 == 1:
        ot_best_home_odds_list.append(ot_all_odds_list[i].text)
    i += 1


# Convert US odds to UK odds
def convert_odds_to_uk(us_odds_list):
    uk_odds_list = []
    for us_odd in us_odds_list:
        if us_odd == 'even':
            uk_odd = 2.0
        elif us_odd == '-':
            uk_odd = 1.0  # Representing a loss of the bet as 100% loss (decimal odds of 1.0)
        else:
            us_odd = int(us_odd)
            if us_odd > 0:
                uk_odd = (us_odd / 100) + 1
            else:
                uk_odd = (100 / abs(us_odd)) + 1
        uk_odds_list.append(round(uk_odd, 2))
    return uk_odds_list

ot_best_away_odds_us = ot_best_away_odds_list.copy()
ot_best_home_odds_us = ot_best_home_odds_list.copy()

ot_best_away_odds_uk = convert_odds_to_uk(ot_best_away_odds_list)
ot_best_home_odds_uk = convert_odds_to_uk(ot_best_home_odds_list)

# Print odds data

print("Away Odds (US):", ot_best_away_odds_us)
print("Home Odds (US):", ot_best_home_odds_us)

print("Away Odds (UK):", ot_best_away_odds_uk)
print("Home Odds (UK):", ot_best_home_odds_uk)



TimeoutException: Message: 


In [ ]:
#odds trader best sites: 


from selenium.common.exceptions import StaleElementReferenceException

ot_all_best_sites = []
ot_best_away_sites = []
ot_best_home_sites = []

# Wait for all elements to be present
ot_all_best_sites = WebDriverWait(driver, 7).until(
    EC.presence_of_all_elements_located((By.XPATH, '//span[@class="subtitle"]'))
)

retry_count = 5
while retry_count > 0:
    try:
        ot_all_best_sites_text = [elem.text for elem in ot_all_best_sites if elem.text.strip() != "ODDS BY"]
        # Only take the sites for the indexes up to the length of ot_home_teams_list
        ot_best_away_sites = [site for site in ot_all_best_sites_text[:2 * len(ot_home_players_list):2] if site.strip()]
        ot_best_home_sites = [site for site in ot_all_best_sites_text[1:2 * len(ot_home_players_list):2] if site.strip()]

        break  # Exit the loop if successful
    except StaleElementReferenceException:
        retry_count -= 1

if retry_count == 0:
    print("Failed to retrieve element text due to StaleElementReferenceException")

# Print the best sites lists
print("Best Away Sites:", ot_best_away_sites)
print("Best Home Sites:", ot_best_home_sites)



In [ ]:
#coverting uk odds from oddstrader into away prob and home prob 

ot_home_probablity_odds = []
for odds in ot_best_home_odds_uk: 
    ot_home_probablity_odds.append(1 /odds)

ot_away_probablity_odds = []
for odds in ot_best_away_odds_uk: 
    ot_away_probablity_odds.append(1 /odds)

print(ot_away_probablity_odds)
print(ot_home_probablity_odds)

In [ ]:
def calc_home_stake(implied_home_prob, implied_away_prob, total_wager):
    total_implied_prob = implied_home_prob + implied_away_prob
    scaling_factor = total_wager / total_implied_prob
    home_stake = implied_home_prob * scaling_factor
    home_stake = round(home_stake, 2)
    
    return home_stake

def calc_away_stake(implied_home_prob, implied_away_prob, total_wager):
    total_implied_prob = implied_home_prob + implied_away_prob
    scaling_factor = total_wager / total_implied_prob
    away_stake = implied_away_prob * scaling_factor
    away_stake = round(away_stake, 2)
    
    return away_stake

#print(calc_home_stake(0.7462686567164178, 0.28169014084507044, 100))


In [ ]:
#calculating optimal percentage profit (guaranteed):

def calc_opt_perc_profit(implied_home_prob, implied_away_prob):
    total_implied_prob = implied_home_prob + implied_away_prob
    scaling_factor = 1 / total_implied_prob
    home_stake = implied_home_prob * scaling_factor
    away_stake = implied_away_prob * scaling_factor
    home_payout = home_stake * (1 / implied_home_prob)
    away_payout = away_stake * (1 / implied_away_prob)
    guaranteed_profit = max(home_payout, away_payout) - 1
    percentage_profit = round(guaranteed_profit * 100, 2)
    return f"{percentage_profit}%"

def calc_dollar_profit(implied_home_prob, implied_away_prob):
    total_implied_prob = implied_home_prob + implied_away_prob
    scaling_factor = 1 / total_implied_prob
    home_stake = implied_home_prob * scaling_factor
    away_stake = implied_away_prob * scaling_factor
    home_payout = home_stake * (1 / implied_home_prob)
    away_payout = away_stake * (1 / implied_away_prob)
    guaranteed_profit = max(home_payout, away_payout) - 1
    dollar_profit = round(guaranteed_profit * 100, 2)
    return f"${dollar_profit}"
  





In [ ]:
#loop through the home and away OT odds to see if any are arb opps: 
    # append all info to new lists if are 

ot_arb_away_player = []
ot_arb_home_player = []
ot_arb_away_odds = []
ot_arb_home_odds = []
ot_arb_away_site = []
ot_arb_home_site = []
ot_arb_away_wager_amount = []
ot_arb_home_wager_amount = []
ot_arb_perc_profit = []
ot_arb_dollar_profit = []

# Ensure both lists have the same length
if len(ot_away_probablity_odds) == len(ot_home_probablity_odds):
    for i in range(len(ot_away_probablity_odds)):
        if ot_away_probablity_odds[i] + ot_home_probablity_odds[i] < 1.1:
            ot_arb_away_player.append(ot_away_players_list[i])
            ot_arb_home_player.append(ot_home_players_list[i])
            ot_arb_away_odds.append(ot_best_away_odds_us[i])
            ot_arb_home_odds.append(ot_best_home_odds_us[i])
            ot_arb_away_site.append(ot_best_away_sites[i])
            ot_arb_home_site.append(ot_best_home_sites[i])
            ot_arb_away_wager_amount.append(calc_away_stake(ot_home_probablity_odds[i], ot_away_probablity_odds[i], 100))
            ot_arb_home_wager_amount.append(calc_home_stake(ot_home_probablity_odds[i], ot_away_probablity_odds[i], 100))
            ot_arb_perc_profit.append(calc_opt_perc_profit(ot_home_probablity_odds[i], ot_away_probablity_odds[i]))
            ot_arb_dollar_profit.append(calc_dollar_profit(ot_home_probablity_odds[i], ot_away_probablity_odds[i])) 
        else: 
            print("No Current Arbitrage Opportunities") 


print(ot_best_away_odds_us) 
print(ot_best_home_odds_us) 




In [ ]:
#making a dict for all the essential arb info 

arb_opps_info = {"Home Player": ot_arb_home_player, "Away Player": ot_arb_away_player, "Home Odds": ot_arb_home_odds, "Away Odds": ot_arb_away_odds, "Home Book": ot_arb_home_site, "Away Book": ot_arb_away_site, "Home wager":ot_arb_home_wager_amount, "Away wager": ot_arb_away_wager_amount, "Profit(%)": ot_arb_perc_profit, "Profit($)": ot_arb_dollar_profit}
print(arb_opps_info)

In [ ]:
#making df for arb opps info
df_arb = pd.DataFrame.from_dict(arb_opps_info)
df_arb


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


def send_email(x):
    MY_ADDRESS = "surebetsignals@outlook.com"
    MY_PASSWORD = "Maddie2013$"
    RECIPIENT_ADDRESS = "dillonk@unc.edu"

    HOST_ADDRESS = "smtp-mail.outlook.com"
    HOST_PORT = 587

    # Set up the SMTP server
    server = smtplib.SMTP(host=HOST_ADDRESS, port=HOST_PORT)
    server.starttls()
    server.login(MY_ADDRESS, MY_PASSWORD)

    # Create the email
    message = MIMEMultipart()
    message["From"] = MY_ADDRESS
    message["To"] = RECIPIENT_ADDRESS
    message["Subject"] = "Arbitrage Opportunity"

# Convert DataFrame to HTML table
    html_table = x.to_html(index=False)

    # Attach the HTML table to the email body
    htmlPart = MIMEText(html_table, 'html')
    message.attach(htmlPart)

    # Send the email
    server.send_message(message)
    server.quit()

    print("email sent successfully")

# Send email if condition is met
def check_condition():
    if ot_away_probablity_odds[i] + ot_home_probablity_odds[i] < 1:
        send_email(df_arb)
    else: 
        print("No current arbitrage opportunities")

if __name__ == "__main__":
    check_condition()

    
